In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Setup and imports
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, DenseNet121, EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
#Load Train/Val/Test sets

# Define paths
base_path = '/content/drive/MyDrive/MLProjects/pneumonia-detection/data/pneumonia_split'
img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    os.path.join(base_path, 'train'), target_size=img_size, batch_size=batch_size, class_mode='binary'
)
val_gen = datagen.flow_from_directory(
    os.path.join(base_path, 'val'), target_size=img_size, batch_size=batch_size, class_mode='binary'
)
test_gen = datagen.flow_from_directory(
    os.path.join(base_path, 'test'), target_size=img_size, batch_size=batch_size, class_mode='binary', shuffle=False
)


Found 2003 images belonging to 2 classes.
Found 429 images belonging to 2 classes.
Found 430 images belonging to 2 classes.


In [4]:
#Define model training function
def build_model(base_model_fn, input_shape=(224, 224, 3)):
  base_model = base_model_fn(weights = 'imagenet', include_top = False, input_shape=input_shape)
  model = Sequential([
      base_model,
      GlobalAveragePooling2D(),
      Dense(1, activation = 'sigmoid')
  ])

  model.compile(optimizer = Adam(1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model


In [5]:
#Train and evaluate models
models = {
    'ResNet50': ResNet50,
    'DenseNet121': DenseNet121,
    'EfficientNetB0': EfficientNetB0
}

results = {}

for name, model_fn in models.items():
  print(f'\nTraining{name}...')
  model = build_model(model_fn)
  start = time.time()
  history = model.fit(train_gen, validation_data = val_gen, epochs = 5, verbose=1)
  duration = time.time() - start
  loss, acc = model.evaluate(test_gen)
  results[name] = {'accuracy': acc, 'loss': loss, 'time': duration}


TrainingResNet50...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 432s 6s/step - accuracy: 0.6658 - loss: 0.6440 - val_accuracy: 0.5338 - val_loss: 0.6923
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 666ms/step - accuracy: 0.9035 - loss: 0.2605 - val_accuracy: 0.4848 - val_loss: 0.7090
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 660ms/step - accuracy: 0.9835 - loss: 0.0742 - val_accuracy: 0.4848 - val_loss: 0.6982
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 664ms/step - accuracy: 0.9927 - loss: 0.0366 - val_accuracy: 0.4848 - val_loss: 0.7174
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 662ms/step - accuracy: 0.9889 - loss: 0.0404 - val_accuracy: 0.5152 - val_loss: 0.8867
14/14 ━━━━━━━━━━━━━━━━━━━━ 88s 7s/step - accuracy: 0.1935 - loss: 1.2815

TrainingDenseNet121...
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 301s 2s/step - accuracy: 0.5886 - loss: 0.8628 - val_accuracy: 0.5291 - val_loss: 0.8223
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 669ms/step - accuracy: 0.8745 - loss: 0.3486 - 

In [7]:
pd.DataFrame(results).T

,accuracy,loss,time
ResNet50,0.483721,0.905236,618.251459
DenseNet121,0.651163,0.842498,469.621567
EfficientNetB0,0.488372,0.714860,359.232129
